In [ ]:
!pip install datetime
!pip install pandas
!pip install numpy
!pip install scipy
!pip install vaderSentiment
!pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
twt_df = pd.read_csv('indextweetsdata.csv')
twt_df['date'] = pd.to_datetime(twt_df['Datetime']).dt.date
twt_df = twt_df.drop(columns = ['Unnamed: 0','Tweet Id','Username','Datetime'])
twt_df

In [ ]:
twt_df['Index_Score'] = ''
twt_df['Index_Neg'] = ''
twt_df['Index_Pos'] = ''
twt_df['Index_Score'] = pd.to_numeric(twt_df['Index_Score'])
twt_df['Index_Neg'] = pd.to_numeric(twt_df['Index_Neg'])
twt_df['Index_Pos'] = pd.to_numeric(twt_df['Index_Pos'])
twt_df 

In [ ]:
analyzer = SentimentIntensityAnalyzer()
for i in range(twt_df.shape[0]):
    text_analyzed = analyzer.polarity_scores(twt_df['Text'][i])
    twt_df['Index_Score'][i] = text_analyzed['compound']
    twt_df['Index_Pos'][i] = text_analyzed['pos']
    twt_df['Index_Neg'][i] = text_analyzed['neg']
    

In [ ]:
twt_df

In [ ]:
twt_df = (twt_df.groupby(['date'])['Index_Score', 'Index_Pos', 'Index_Neg'].mean()).reset_index()
twt_df 

In [ ]:
stock_df = pd.read_csv('index1data.csv')
stock_df['date'] = pd.to_datetime(stock_df['Date']).dt.date
stock_df = stock_df.drop(['Date'], axis=1)
stock_df['DR'] = np.log((stock_df['Close']/ stock_df['Close'].shift(1)))
stock_df['DV7'] = ''
stock_df['DV3'] = ''

for i in range(7):
    col_name = 'shift'+str(i)
    stock_df[col_name] = stock_df['Close'].shift(i)
    
for i in range(stock_df.shape[0]):
    stock_df['DV7'][i] = np.sqrt(np.var([stock_df[('shift'+str(j))][i] for j in range(7)]))
    stock_df['DV3'][i] = np.sqrt(np.var([stock_df[('shift'+str(j))][i] for j in range(3)]))
    
stock_df['DV7-1'] = stock_df['DV7'].shift(-1)
stock_df['DV3-1'] = stock_df['DV3'].shift(-1)
    
stock_df

In [ ]:
main_df = twt_df[['date','Index_Score']]
main_df2 = twt_df[['date','Index_Pos']]
main_df3 = twt_df [['date','Index_Neg']]

In [ ]:
main_merged_df = pd.merge(stock_df, main_df, on='date',how='inner')
main_merged_df = main_merged_df[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Score']]
main_merged_df['DV7'] = pd.to_numeric(main_merged_df['DV7'])
main_merged_df['DV7-1'] = pd.to_numeric(main_merged_df['DV7-1'])
main_merged_df['DV3'] = pd.to_numeric(main_merged_df['DV3'])
main_merged_df['DV3-1'] = pd.to_numeric(main_merged_df['DV3-1'])

main_merged_df2 = pd.merge(stock_df, main_df2, on='date',how='inner')
main_merged_df2 = main_merged_df2[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Pos']]
main_merged_df2['DV7'] = pd.to_numeric(main_merged_df2['DV7'])
main_merged_df2['DV7-1'] = pd.to_numeric(main_merged_df2['DV7-1'])
main_merged_df2['DV3'] = pd.to_numeric(main_merged_df2['DV3'])
main_merged_df2['DV3-1'] = pd.to_numeric(main_merged_df2['DV3-1'])

main_merged_df3 = pd.merge(stock_df, main_df3, on='date',how='inner')
main_merged_df3 = main_merged_df3[['Close','DR','DV3','DV7','DV3-1','DV7-1','Index_Neg']]
main_merged_df3['DV7'] = pd.to_numeric(main_merged_df3['DV7'])
main_merged_df3['DV7-1'] = pd.to_numeric(main_merged_df3['DV7-1'])
main_merged_df3['DV3'] = pd.to_numeric(main_merged_df3['DV3'])
main_merged_df3['DV3-1'] = pd.to_numeric(main_merged_df3['DV3-1'])

In [ ]:
import numpy as np
from scipy.stats import pearsonr

print("Index_Score, Price:", pearsonr(main_merged_df['Index_Score'],main_merged_df['Close']))
print("Index_Score, DR:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DR']))
print("Index_Score, DV7:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV7']))
print("Index_Score, DV7-1:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV7-1']))
print("Index_Score, DV3:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV3']))
print("Index_Score, DV3-1:", pearsonr(main_merged_df['Index_Score'],main_merged_df['DV3-1']))

In [ ]:
print("Index_Pos, Price:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['Close']))
print("Index_Pos, DR:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DR']))
print("Index_Pos, DV7:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV7']))
print("Index_Pos, DV7-1:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV7-1']))
print("Index_Pos, DV3:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV3']))
print("Index_Pos, DV3-1:", pearsonr(main_merged_df2['Index_Pos'],main_merged_df2['DV3-1']))

In [ ]:
print("Index_Neg, Price:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['Close']))
print("Index_Neg, DR:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DR']))
print("Index_Neg, DV7:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV7']))
print("Index_Neg, DV7-1:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV7-1']))
print("Index_Neg, DV3:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV3']))
print("Index_Neg, DV3-1:", pearsonr(main_merged_df3['Index_Neg'],main_merged_df3['DV3-1']))

In [ ]:
from statsmodels.tsa.stattools import adfuller

print("ADF Close", adfuller(main_merged_df['Close']))
print("ADF DR", adfuller(main_merged_df['DR']))
print("ADF DV7", adfuller(main_merged_df['DV7']))
print("ADF DV7-1", adfuller(main_merged_df['DV7-1']))
print("ADF DV3", adfuller(main_merged_df['DV3']))
print("ADF DV3-1", adfuller(main_merged_df['DV3-1']))
print("ADF Index_Score", adfuller(main_merged_df['Index_Score']))
print("ADF Index_Pos", adfuller(main_merged_df2['Index_Pos']))
print("ADF Index_Neg", adfuller(main_merged_df3['Index_Neg']))

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

print("IS - DV7", grangercausalitytests(main_merged_df[['Index_Score', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df[['DV7','Index_Score']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df[['Index_Score', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df[['DV7-1','Index_Score']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df[['Index_Score', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df[['DV3','Index_Score']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df[['Index_Score', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df[['DV3-1','Index_Score']], maxlag=[1]))
print()

In [ ]:
print("IS - DV7", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df2[['DV7','Index_Pos']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df2[['DV7-1','Index_Pos']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df2[['DV3','Index_Pos']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df2[['Index_Pos', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df2[['DV3-1','Index_Pos']], maxlag=[1]))
print()

In [ ]:
print("IS - DV7", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV7']], maxlag=[1]))
print()
print("DV7 - IS", grangercausalitytests(main_merged_df3[['DV7','Index_Neg']], maxlag=[1]))
print()
print("IS - DV7-1", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV7-1']], maxlag=[1]))
print()
print("DV7 - IS-1", grangercausalitytests(main_merged_df3[['DV7-1','Index_Neg']], maxlag=[1]))
print()
print("IS - DV3", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV3']], maxlag=[1]))
print()
print("DV3 - IS", grangercausalitytests(main_merged_df3[['DV3','Index_Neg']], maxlag=[1]))
print()
print("IS - DV3-1", grangercausalitytests(main_merged_df3[['Index_Neg', 'DV3-1']], maxlag=[1]))
print()
print("DV3 - IS-1", grangercausalitytests(main_merged_df3[['DV3-1','Index_Neg']], maxlag=[1]))
print()